In [1]:
!pip install efficientnet
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from efficientnet.keras import EfficientNetB5
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from sklearn.metrics import classification_report

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.9 MB/s eta 0:00:00


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
# Set the path to the dataset
data_path = '/kaggle/input/garbage-classification-v2/Garbage Classification'

def load_images(path, target_shape):
    X = []
    y = []
    for folder in os.listdir(path):
        if os.path.isdir(os.path.join(path, folder)):
            folder_path = os.path.join(path, folder)
            for image_file in os.listdir(folder_path):
                image_path = os.path.join(folder_path, image_file)
                try:
                    image = cv2.imread(image_path)
                    if image is None:
                        continue  # Skip non-image files
                    resized_image = cv2.resize(image, target_shape[:2])
                    X.append(resized_image)
                    y.append(folder)
                except Exception as e:
                    print(f"Error processing image: {image_path} - {e}")
    X = np.array(X)
    y = np.array(y)
    return X, y


# Set the target shape for input images
target_shape = (90, 90, 3)

# Load and preprocess the data
X, y = load_images(data_path, target_shape)
num_classes = len(np.unique(y))
y = to_categorical(LabelEncoder().fit_transform(y))

# Split the dataset into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
    

libpng warning: Out of place sRGB chunk
libpng warning: Out of place sRGB chunk
libpng warning: Out of place sRGB chunk
libpng warning: Out of place sRGB chunk


In [4]:
# Construct the EfficientNetB5 model
base_model = EfficientNetB5(weights='imagenet', include_top=False, input_shape=target_shape, pooling='max')

model = Sequential([
    base_model,
    BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001),
    Dense(256, kernel_regularizer=regularizers.l2(l=0.016), activity_regularizer=regularizers.l1(0.006),
          bias_regularizer=regularizers.l1(0.006), activation='relu'),
    Dropout(rate=0.45, seed=123),
    Dense(num_classes, activation='softmax')
])

model.compile(Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

115515256/115515256 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnet-b5 (Functional  (None, 2048)             28513520  
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 2048)             8192      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                       

In [5]:
import tensorflow as tf

class MyCallback(tf.keras.callbacks.Callback):
    def __init__(self, model, patience, stop_patience, threshold, factor, batches, epochs, ask_epoch):
        super(MyCallback, self).__init__()
        self.model = model
        self.patience = patience
        self.stop_patience = stop_patience
        self.threshold = threshold
        self.factor = factor
        self.batches = batches
        self.epochs = epochs
        self.ask_epoch = ask_epoch
        self.ask_epoch_initial = ask_epoch

        self.count = 0
        self.stop_count = 0
        self.best_epoch = 1
        self.initial_lr = float(tf.keras.backend.get_value(model.optimizer.lr))
        self.highest_tracc = 0.0
        self.lowest_vloss = np.inf
        self.best_weights = self.model.get_weights()
        self.initial_weights = self.model.get_weights()

    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.batches == 0 and epoch > 0:
            tr_acc = logs.get('accuracy')
            v_loss = logs.get('val_loss')

            if tr_acc > self.highest_tracc:
                self.highest_tracc = tr_acc

            if v_loss < self.lowest_vloss:
                self.lowest_vloss = v_loss
                self.best_epoch = epoch + 1
                self.best_weights = self.model.get_weights()
                self.count = 0
            else:
                self.count += 1

            if tr_acc >= self.threshold:
                self.ask_epoch -= 1

            if self.count == self.patience and self.ask_epoch > 0:
                print("\nEpoch %d: Accuracy threshold reached. Decreasing learning rate." % (epoch + 1))
                old_lr = float(tf.keras.backend.get_value(self.model.optimizer.lr))
                new_lr = old_lr * self.factor
                tf.keras.backend.set_value(self.model.optimizer.lr, new_lr)
                print("Learning rate decreased from %f to %f" % (old_lr, new_lr))
                self.count = 0

            if self.count == self.patience and self.ask_epoch == 0:
                self.stop_count += 1
                if self.stop_count == self.stop_patience:
                    print("\nTraining stopped at epoch %d" % (epoch + 1))
                    self.model.stop_training = True
                else:
                    print("\nEpoch %d: Learning rate adjustment limit reached. Restoring best weights." % (epoch + 1))
                    self.model.set_weights(self.best_weights)
                    self.count = 0

In [6]:
# Define the callbacks
checkpoint = ModelCheckpoint('/kaggle/working/best_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
my_callback = MyCallback(model=model, patience=10, stop_patience=5, threshold=0.9, factor=0.1, batches=10, epochs=100, ask_epoch=10)

# Train the model
history = model.fit(X_train, y_train, batch_size=8, epochs=20, validation_data=(X_val, y_val), callbacks=[checkpoint, my_callback])

# Load the best weights
model.load_weights('/kaggle/working/best_model.h5')

Epoch 1/20


2023-07-14 17:12:38.292209: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/efficientnet-b5/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1979/1979 [==============================] - ETA: 0s - loss: 5.3081 - accuracy: 0.6395
Epoch 1: val_accuracy improved from -inf to 0.84764, saving model to /kaggle/working/best_model.h5
1979/1979 [==============================] - 305s 101ms/step - loss: 5.3081 - accuracy: 0.6395 - val_loss: 2.1692 - val_accuracy: 0.8476
Epoch 2/20
1979/1979 [==============================] - ETA: 0s - loss: 1.5994 - accuracy: 0.7914
Epoch 2: val_accuracy improved from 0.84764 to 0.89596, saving model to /kaggle/working/best_model.h5
1979/1979 [==============================] - 198s 100ms/step - loss: 1.5994 - accuracy: 0.7914 - val_loss: 0.9184 - val_accuracy: 0.8960
Epoch 3/20
1979/1979 [==============================] - ETA: 0s - loss: 0.9269 - accuracy: 0.8521
Epoch 3: val_accuracy improved from 0.89596 to 0.93348, saving model to /kaggle/working/best_model.h5
1979/1979 [==============================] - 200s 101ms/step - loss: 0.9269 - accuracy: 0.8521 - val_loss: 0.6324 - val_accuracy: 0.9335
Epo

In [8]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)


138/138 [==============================] - 6s 43ms/step - loss: 0.3485 - accuracy: 0.9495
Test loss: 0.3485081195831299
Test accuracy: 0.9495110511779785


In [9]:
# Generate predictions for the test set
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test, axis=1)

# Convert integer labels to class names
class_names = sorted(os.listdir(data_path))
y_true_names = np.array(class_names)[y_true_labels]
y_pred_names = np.array(class_names)[y_pred_labels]

# Generate and print the classification report
report = classification_report(y_true_names, y_pred_names)
print("Classification Report:")
print(report)


138/138 [==============================] - 9s 38ms/step
Classification Report:
              precision    recall  f1-score   support

     battery       0.96      0.92      0.94       185
  biological       0.94      0.95      0.94       235
   cardboard       0.96      0.95      0.95       389
     clothes       0.99      0.98      0.98      1065
       glass       0.97      0.93      0.95       792
       metal       0.84      0.95      0.89       346
       paper       0.92      0.96      0.94       421
     plastic       0.94      0.92      0.93       445
       shoes       0.95      0.94      0.95       387
       trash       0.95      0.92      0.94       132

    accuracy                           0.95      4397
   macro avg       0.94      0.94      0.94      4397
weighted avg       0.95      0.95      0.95      4397

